In [1]:
import h5py 
import pylab
import numpy as np
import timeit
import os
import matplotlib.pyplot as plt

from pycbc.waveform import get_td_waveform
from pycbc.detector import Detector

import pycbc.noise
import pycbc.psd

/anaconda3/envs/anacondapy2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
PyCBC.libutils: pkg-config call failed, setting NO_PKGCONFIG=1


In [2]:
fs = 4096 #sample rate
f_low = 30 #GW數據的minimum freq
apx = 'IMRPhenomD' #近似法
detector_name = 'L1' #選取模擬的detector
interval_duration = 64 #數據時間長
lenT = int(fs*interval_duration) #總數據點數
minpeak = 3.126246936618704e-21*3 #inc=pi/2, coa=pi/2, m1=m2=1.4 #1.209761294218286e-20
data_dir = '/Volumes/Mac/GW_glitch/' #存放模擬數據的資料夾

# Simulate glitch

In [3]:
f_psdphase = h5py.File('./glitch_psd_with_phase_L1','r') #讀取glitch的psd
psd_g_L1 = f_psdphase['psd'][...].copy()
Phase_g_L1 = f_psdphase['phase'][...].copy()
fs_g_L1 = pycbc.types.frequencyseries.FrequencySeries(psd_g_L1, 1) #將type從array轉為FrequencySeries
f_psdphase.close()

In [4]:
def glitch_generator():
    fs_g_L1_s = pycbc.noise.gaussian.frequency_noise_from_psd(fs_g_L1) #模擬glitch
    Hn_g_L1_oneside_s = np.sqrt(fs_g_L1_s/2)*fs * np.exp(1j*Phase_g_L1)
    Hn_g_L1_s = np.concatenate((Hn_g_L1_oneside_s, Hn_g_L1_oneside_s[1:][::-1])) #轉為two-side psd
    glitch_L1_s = np.fft.ifft(Hn_g_L1_s, n=fs) #轉換回時域
    return np.real(glitch_L1_s)

# Generate GW data with glitch

In [5]:
def my_random_homo_2Dsphere():
    check = 1
    
    while check:
        x = np.random.rand(3)*2-1 #產生隨機xyz座標參數
        if np.dot(x,x)<1: #取單位圓內座標
            check=0
    
    r = np.sqrt(np.dot(x,x)) #計算長度r=sqrt(x^2+y^2+z^2)
    r_xy = np.sqrt(np.dot(x[:-1],x[:-1])) #計算r*sin(theta)=sqrt(x^2+y^2)
    theta = np.arccos(x[2]/r) #因為r*cos(theta)=z, 所以theta=arccos(z/r)
    phi = np.arccos(x[0]/r_xy) #因為r*sin(theta)*cos(phi)=x, 所以phi=arccos(x/(r*sin(theta)))
    
    return theta, phi

In [6]:
def stimulate_ndata_generator(mode, factor1, factor2, n, lamb): #產生n筆的數據，factor1控制White Noise大小，factor2控制glitch大小
    start_time = timeit.default_timer()
    stop_time_b = timeit.default_timer()
    
    f = h5py.File(data_dir+'{}_{}/{}/{}_{}_waveform.h5'.format(factor1, factor2, mode, lamb, n),'a')
    
    Nlist = np.random.poisson(lamb,size=(n,)) #每筆數據的glitch數目
    for m in range(n):
        datam = f.create_group('data{:0>4d}'.format(m+1))
        #generate GW
        source_distance=1
        m1 = np.random.rand(1)[0]*1.6+1.4
        m2 = np.random.rand(1)[0]*1.6+1.4
        inc,coa = my_random_homo_2Dsphere()
        coa = coa + np.pi/2
        hp, hc = get_td_waveform(approximant=apx,
                             distance=source_distance,
                             mass1=m1,
                             mass2=m2,
                             spin1z=0,
                             spin2z=0,
                             inclination=inc,
                             coa_phase=coa,
                             delta_t=1.0/fs,
                             f_lower=f_low)
        det = Detector(detector_name)
        # Choose a GPS end time, sky location, and polarization phase for the merger
        # NOTE: Right ascension and polarization phase runs from 0 to 2pi
        # Declination runs from pi/2. to -pi/2 with the poles at pi/2. and -pi/2.
        declination, right_ascension = my_random_homo_2Dsphere()
        declination = declination - np.pi/2.0
        polarization = np.random.rand(1)[0]*np.pi*2
        
        signal = det.project_wave(hp, hc,  right_ascension, declination, polarization)
        lenl = len(signal)
        signal = signal[int(lenl*0.2):] #去掉GW前面凸起的部分
        lenL = len(signal)
        
        strain_ = np.zeros(lenT+lenL)
        gwposition = np.random.randint(lenT)
        strain_[gwposition : gwposition+lenL] = signal
        strain = strain_[int(0.5*lenL) : int(0.5*lenL)+lenT] #截取中間0~64s 的數據
        
        gwidx = np.where(strain!=0.)
        gwstart = np.min(gwidx) #GW訊號開始
        gwend = np.max(gwidx) #GW訊號結束
        
        #generate WN
        noise = np.random.normal(0, minpeak*factor1, lenT)
        
        data = datam.create_dataset('TS', (3, lenT))
        data[0] = strain
        data[1] = noise
        
        #generate glitch
        noverlap = 0
        if Nlist[m] != 0:
            glitch = np.zeros(lenT)
            glitchpositions = np.random.randint((lenT-fs-fs/2), size=Nlist[m]) 
            #為了之後spec要cut掉最後一個time bin，所以只允許在(0s, lenTs-0.5s)的範圍
            for glitchposition in glitchpositions:
                mult = np.random.rand()*7+3
                glitch[glitchposition : glitchposition+fs] += glitch_generator()/60/factor2*minpeak*factor1*mult
                
                if gwstart-fs<=glitchposition<=gwend:
                    noverlap+=1
            data[2] = glitch 
        else:
            data[2] = np.zeros(lenT)
        
        parameter=[]
        parameter.append(m1)
        parameter.append(m2)
        parameter.append(inc)
        parameter.append(coa)
        parameter.append(declination)
        parameter.append(right_ascension)
        parameter.append(polarization)
        parameter.append(Nlist[m])
        parameter.append(noverlap)
        datam.create_dataset('parameter', data=np.array(parameter)) #紀錄GW的parameter
        if (m+1)%500==0:
            stop_time_a = timeit.default_timer()
            print m+1, 'th data has done, duration(s) =', stop_time_a - stop_time_b
            stop_time_b = stop_time_a*1
            
    stop_time_a = timeit.default_timer()
    print 'job finish'
    print ' total duration(s) =', stop_time_a - start_time
    f.close()
    

In [7]:
stimulate_ndata_generator('train', 1, 1, 9000, 2)

500 th data has done, duration(s) = 1795.40550208
1000 th data has done, duration(s) = 3198.59746194
1500 th data has done, duration(s) = 1794.91632605
2000 th data has done, duration(s) = 1735.91880798
2500 th data has done, duration(s) = 1738.02089691
3000 th data has done, duration(s) = 1781.42162108
3500 th data has done, duration(s) = 1762.02469492
4000 th data has done, duration(s) = 1702.84840608
4500 th data has done, duration(s) = 1735.47180009
5000 th data has done, duration(s) = 1807.38214588
5500 th data has done, duration(s) = 1754.39397693
6000 th data has done, duration(s) = 1725.98503709
6500 th data has done, duration(s) = 1775.70567703
7000 th data has done, duration(s) = 1764.44723296
7500 th data has done, duration(s) = 1762.22597289
8000 th data has done, duration(s) = 1830.40617609
8500 th data has done, duration(s) = 1769.01529193
9000 th data has done, duration(s) = 1726.64454103
job finish
 total duration(s) = 33160.8651719
